In [1]:
import statsmodels as sm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
PATH = "data\\covid_19_data.csv"
PATH_CONFIRM = "data\\time_series_covid_19_confirmed.csv"
PATH_DEATHS = "data\\time_series_covid_19_deaths.csv"
PATH_RECOVERED = "data\\time_series_covid_19_recovered.csv"

In [3]:
data = pd.read_csv(PATH)
confirmed_df = pd.read_csv(PATH_CONFIRM)
deaths_df = pd.read_csv(PATH_DEATHS)
recovered_df = pd.read_csv(PATH_RECOVERED)

In [4]:
data

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
77462,77463,08/01/2020,Zaporizhia Oblast,Ukraine,2020-08-02 04:34:47,800.0,21.0,597.0
77463,77464,08/01/2020,Zeeland,Netherlands,2020-08-02 04:34:47,867.0,69.0,0.0
77464,77465,08/01/2020,Zhejiang,Mainland China,2020-08-02 04:34:47,1270.0,1.0,1268.0
77465,77466,08/01/2020,Zhytomyr Oblast,Ukraine,2020-08-02 04:34:47,1896.0,37.0,1485.0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77467 entries, 0 to 77466
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SNo              77467 non-null  int64  
 1   ObservationDate  77467 non-null  object 
 2   Province/State   51018 non-null  object 
 3   Country/Region   77467 non-null  object 
 4   Last Update      77467 non-null  object 
 5   Confirmed        77467 non-null  float64
 6   Deaths           77467 non-null  float64
 7   Recovered        77467 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 4.7+ MB


In [6]:
des = data.describe()
des.astype(int)

,SNo,Confirmed,Deaths,Recovered
count,77467,77467,77467,77467
mean,38734,11807,600,5745
std,22362,35795,2569,32337
min,1,0,0,0
25%,19367,132,1,1
50%,38734,1154,21,167
75%,58100,6198,196,1842
max,77467,552318,41584,1461885


Chuẩn hóa format ngày giờ

In [7]:
def get_time(s):
    if "T" in s:
        return s.split("T")[1]
    else:
        t =  s.split()[1]
        if t.count(":") == 1:
            t += ":00"
        return t
data["Update Time"] = data["Last Update"].apply(get_time)

In [8]:
def get_date(s):
    if "T" in s:
        return s.split("T")[0]
    else:
        t =  s.split()[0]
        return t
data["Update Date"] = data["Last Update"].apply(get_date)

In [9]:
data

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Update Time,Update Date
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0,17:00:00,1/22/2020
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0,17:00:00,1/22/2020
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0,17:00:00,1/22/2020
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0,17:00:00,1/22/2020
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0,17:00:00,1/22/2020
...,...,...,...,...,...,...,...,...,...,...
77462,77463,08/01/2020,Zaporizhia Oblast,Ukraine,2020-08-02 04:34:47,800.0,21.0,597.0,04:34:47,2020-08-02
77463,77464,08/01/2020,Zeeland,Netherlands,2020-08-02 04:34:47,867.0,69.0,0.0,04:34:47,2020-08-02
77464,77465,08/01/2020,Zhejiang,Mainland China,2020-08-02 04:34:47,1270.0,1.0,1268.0,04:34:47,2020-08-02
77465,77466,08/01/2020,Zhytomyr Oblast,Ukraine,2020-08-02 04:34:47,1896.0,37.0,1485.0,04:34:47,2020-08-02


In [10]:
isna_df = data[data.isna()==True]

In [11]:
isna_df

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Update Time,Update Date
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
77462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77464,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
confirmed_df

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,35928,35981,36036,36157,36263,36368,36471,36542,36675,36710
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,4466,4570,4637,4763,4880,4997,5105,5197,5276,5396
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,25484,26159,26764,27357,27973,28615,29229,29831,30394,30950
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,889,897,897,897,907,907,918,922,925,925
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,851,880,916,932,950,1000,1078,1109,1148,1164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,NaN,Sao Tome and Principe,0.186400,6.613100,0,0,0,0,0,0,...,749,860,862,863,865,867,868,870,871,874
262,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,1654,1674,1674,1681,1691,1703,1711,1726,1728,1730
263,NaN,Comoros,-11.645500,43.333300,0,0,0,0,0,0,...,340,340,340,354,354,354,378,378,378,386
264,NaN,Tajikistan,38.861000,71.276100,0,0,0,0,0,0,...,7060,7104,7150,7192,7235,7276,7320,7366,7409,7451


In [13]:
deaths_df

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,1211,1225,1248,1259,1269,1270,1271,1271,1272,1283
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,123,128,134,138,144,148,150,154,157,161
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,1124,1136,1146,1155,1163,1174,1186,1200,1210,1223
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,52,52,52,52,52,52,52,52,52,52
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,33,35,39,40,41,47,48,51,52,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,NaN,Sao Tome and Principe,0.186400,6.613100,0,0,0,0,0,0,...,14,14,14,14,14,14,15,15,15,15
262,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,461,469,474,479,483,484,485,487,493,494
263,NaN,Comoros,-11.645500,43.333300,0,0,0,0,0,0,...,7,7,7,7,7,7,7,7,7,7
264,NaN,Tajikistan,38.861000,71.276100,0,0,0,0,0,0,...,58,58,59,59,60,60,60,60,60,60


In [14]:
recovered_df

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,24550,24602,24793,25180,25198,25358,25389,25471,25509,25509
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,2523,2608,2637,2682,2745,2789,2830,2883,2952,2961
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,17369,17369,18076,18088,18837,19233,19592,20082,20537,20988
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,803,803,803,803,803,803,804,806,807,807
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,236,241,242,242,242,266,301,395,437,460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,NaN,Sao Tome and Principe,0.186400,6.613100,0,0,0,0,0,0,...,604,610,662,696,734,759,766,771,778,782
249,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,762,779,780,797,833,840,849,856,862,862
250,NaN,Comoros,-11.645500,43.333300,0,0,0,0,0,0,...,324,324,324,328,328,328,330,330,330,330
251,NaN,Tajikistan,38.861000,71.276100,0,0,0,0,0,0,...,5793,5851,5906,5970,6028,6065,6103,6151,6193,6233


In [15]:
confirmed_df.columns[4:]

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20',
       ...
       '7/23/20', '7/24/20', '7/25/20', '7/26/20', '7/27/20', '7/28/20',
       '7/29/20', '7/30/20', '7/31/20', '8/1/20'],
      dtype='object', length=193)

In [16]:
data["SNo"]

0            1
1            2
2            3
3            4
4            5
         ...  
77462    77463
77463    77464
77464    77465
77465    77466
77466    77467
Name: SNo, Length: 77467, dtype: int64